In [15]:
from elasticsearch import Elasticsearch
import requests
import tiktoken

In [8]:


# Initialize Elasticsearch client
es = Elasticsearch("http://localhost:9200")

# Define index mapping
index_mapping = {
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

# Create index with mapping
index_name = "course-questions"
es.indices.create(index=index_name, body=index_mapping, ignore=[400])

# Fetch and process documents
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

# Index documents
for i, doc in enumerate(documents):
    es.index(index=index_name, id=i, document=doc)

print(f"Indexed {len(documents)} documents.")

/tmp/ipykernel_15241/3970202474.py:21: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index=index_name, body=index_mapping, ignore=[400])


Indexed 948 documents.


In [9]:
es

<Elasticsearch(['http://localhost:9200'])>

In [10]:

query = {
    "query": {
        "multi_match": {
            "query": "How do I execute a command in a running docker container?",
            "fields": ["question^4", "text"],
            "type": "best_fields"
        }
    }
}

response = es.search(index="course-questions", body=query)

# Print the top result and its score
top_hit = response['hits']['hits'][0]
print(f"Top result score: {top_hit['_score']}")
print(f"Question: {top_hit['_source']['question']}")
print(f"Text: {top_hit['_source']['text'][:200]}...")  # Print first 200 characters of text

Top result score: 84.341125
Question: How do I debug a docker container?
Text: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a ...


In [12]:
query = {
    "size": 3,
    "query": {
        "bool": {
            "must": [
                {
                    "multi_match": {
                        "query": "How do I execute a command in a running docker container?",
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            ]
        }
    }
}

response = es.search(index="course-questions", body=query)

# Print the questions from the results
for i, hit in enumerate(response['hits']['hits'], 1):
    print(f"{i}. {hit['_source']['question']}")

# Print the 3rd question specifically
if len(response['hits']['hits']) >= 3:
    print("\nThe 3rd question is:")
    print(response['hits']['hits'][2]['_source']['question'])
else:
    print("\nThere are fewer than 3 results.")

1. How do I debug a docker container?
2. How do I copy files from my local machine to docker container?
3. How do I copy files from a different folder into docker container’s working directory?

The 3rd question is:
How do I copy files from a different folder into docker container’s working directory?


In [13]:
query = {
    "size": 3,
    "query": {
        "bool": {
            "must": [
                {
                    "multi_match": {
                        "query": "How do I execute a command in a running docker container?",
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            ]
        }
    }
}

response = es.search(index="course-questions", body=query)

context_template = """
Q: {question}
A: {text}
""".strip()

context = "\n\n".join([
    context_template.format(
        question=hit['_source']['question'],
        text=hit['_source']['text']
    )
    for hit in response['hits']['hits']
])

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

question = "How do I execute a command in a running docker container?"
prompt = prompt_template.format(question=question, context=context)

prompt_length = len(prompt)
print(f"The length of the resulting prompt is: {prompt_length}")

The length of the resulting prompt is: 1462


In [16]:
# Calculate tokens
encoding = tiktoken.encoding_for_model("gpt-4")
tokens = encoding.encode(prompt)
token_count = len(tokens)

print(f"The number of tokens in the prompt is: {token_count}")

The number of tokens in the prompt is: 325
